In [2]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict

## Configuration 

In [3]:
hi_df = pd.DataFrame()
hi_df = pd.read_csv( 'file/sample_output/healthinformation.csv') 
#data_df = data_df.to_dict(orient='records')
hi_df

,_id,_index_map,formid,answers.height,answers.bloodtype,answers.identification.number.code,answers.identification.type,answers.identification.number.pin,answers.income,answers.bp1_systolic
0,aQMjmsDcwMbPs5NyhqQPUZ,NaN,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120
1,DexsDsw39ss5NyhqQPUP,NaN,DexsDsw39ss5NyhqQPUP,23,B,2033.2,SSS,333,80.0,201
2,aQMjmsDcwMbPs5NyhqQPUZ,NaN,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120


In [4]:
def set_for_keys(my_dict, key_arr, val):

    current = my_dict
    for i in range(len(key_arr)):
        key = key_arr[i]
        if key not in current:
            if i==len(key_arr)-1:
                current[key] = val
            else:
                current[key] = {}
        else:
            if type(current[key]) is not dict:
                print("not compatible with key structure requested")
                raise ValueError("Dictionary key already occupied")

        current = current[key]

    return my_dict

def to_formatted_json(df, sep="."):
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            
            parsed_row = set_for_keys(parsed_row, keys, val)
        result.append(parsed_row)

    
    return result

def to_formatted_json_test(df, sep="."):
    result = []
    for _, row in df.iterrows():
        parsed_row = {}
        for idx, val in row.iteritems():
            keys = idx.split(sep)
            
            parsed_row = set_for_keys(parsed_row, keys, val)
            
            print('keys: ' , keys)
            print('values: ' , val)
            print('idx: ' , idx)
            print('----------')
            #print('parse_row: ',parsed_row)
        print(parsed_row)
        print('-----parsed_row-----')
        result.append(parsed_row)
        
    
    return result

#Where df was parsed from json-dict using json_normalize

In [5]:
#hi_df = to_formatted_json(data_df, sep=".")
#hi_df

## File loader

In [6]:
fdh_df  = pd.DataFrame()
fdh_df  = pd.read_csv( 'file/sample_output/answers.familydiseasehistory.csv') 
#data_df = data_df.to_dict(orient='records')
fdh_df = fdh_df.sort_values(['_id','_index_map'])
fdh_df

,_id,_index_map,answers.familydiseasehistory
3,DexsDsw39ss5NyhqQPUP,0,Head Cancer
4,DexsDsw39ss5NyhqQPUP,1,Stomach Cancer
0,aQMjmsDcwMbPs5NyhqQPUZ,0,"Respiratory ,disease"
5,aQMjmsDcwMbPs5NyhqQPUZ,0,"Respiratory ,disease"
1,aQMjmsDcwMbPs5NyhqQPUZ,1,Brain Cancer
6,aQMjmsDcwMbPs5NyhqQPUZ,1,Brain Cancer
2,aQMjmsDcwMbPs5NyhqQPUZ,2,Stomach Cancer
7,aQMjmsDcwMbPs5NyhqQPUZ,2,Stomach Cancer


## GROUP BY IF FILE IS ALL ARRAY

In [7]:
fdh_df = fdh_df.groupby('_id').agg(lambda x: x.tolist())
fdh_df = fdh_df.reset_index()
fdh_df

,_id,_index_map,answers.familydiseasehistory
0,DexsDsw39ss5NyhqQPUP,"[0, 1]","[Head Cancer, Stomach Cancer]"
1,aQMjmsDcwMbPs5NyhqQPUZ,"[0, 0, 1, 1, 2, 2]","[Respiratory ,disease, Respiratory ,disease, B..."


In [8]:
fdh_df = fdh_df.drop(['_index_map'], axis=1)
fdh_df

,_id,answers.familydiseasehistory
0,DexsDsw39ss5NyhqQPUP,"[Head Cancer, Stomach Cancer]"
1,aQMjmsDcwMbPs5NyhqQPUZ,"[Respiratory ,disease, Respiratory ,disease, B..."


In [9]:
merge_join = pd.merge(hi_df,fdh_df, left_on='_id', right_on='_id', how='left')
merge_join = merge_join.drop(['_id'], axis=1).drop(['_index_map'], axis=1).replace(np.nan,'')
merge_join


,formid,answers.height,answers.bloodtype,answers.identification.number.code,answers.identification.type,answers.identification.number.pin,answers.income,answers.bp1_systolic,answers.familydiseasehistory
0,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120,"[Respiratory ,disease, Respiratory ,disease, B..."
1,DexsDsw39ss5NyhqQPUP,23,B,2033.2,SSS,333,80.0,201,"[Head Cancer, Stomach Cancer]"
2,aQMjmsDcwMbPs5NyhqQPUZ,16,A,1033.2,SSS,201,60.0,120,"[Respiratory ,disease, Respiratory ,disease, B..."


## JOIN SHOULD BE IN THE PERSISTENT DATABASE ALREADY

In [10]:
add_df  = pd.DataFrame()
add_df  = pd.read_csv( 'file/sample_output/answers.contactnumber.csv') 
#add_df = add_df.sort_values(['_id','_index_map'])
add_df

,_id,_index_map,answers.contactnumber.type
0,aQMjmsDcwMbPs5NyhqQPUZ,0,Landline
1,aQMjmsDcwMbPs5NyhqQPUZ,1,Mobile
2,DexsDsw39ss5NyhqQPUP,0,Landline
3,DexsDsw39ss5NyhqQPUP,1,Mobile
4,aQMjmsDcwMbPs5NyhqQPUZ,0,Landline
5,aQMjmsDcwMbPs5NyhqQPUZ,1,Mobile


In [11]:
add_street_df  = pd.DataFrame()
add_street_df = pd.read_csv( 'file/sample_output/answers.contactnumber.ext.csv') 
add_street_df = add_street_df.sort_values(['_id','_index_map'])
#add_street_df = add_street_df.drop(['_id'], axis=1).drop(['_index_map'], axis=1).replace(np.nan,'')
#add_street_df = add_street_df.drop(['_index_map'], axis=1).replace(np.nan,'')
add_street_df 

,_id,_index_map,answers.contactnumber.ext.number,answers.contactnumber.ext.trunkline
3,DexsDsw39ss5NyhqQPUP,0.0,9,Reception
4,DexsDsw39ss5NyhqQPUP,0.1,5,Operator
5,DexsDsw39ss5NyhqQPUP,1.0,23,Mobile Trunkline
0,aQMjmsDcwMbPs5NyhqQPUZ,0.0,2,Reception
6,aQMjmsDcwMbPs5NyhqQPUZ,0.0,2,Reception
1,aQMjmsDcwMbPs5NyhqQPUZ,1.0,2,Mobile Trunkline
7,aQMjmsDcwMbPs5NyhqQPUZ,1.0,2,Mobile Trunkline
2,aQMjmsDcwMbPs5NyhqQPUZ,1.1,3,Operator
8,aQMjmsDcwMbPs5NyhqQPUZ,1.1,3,Operator


## Convert object into array of object

In [12]:
format_json = to_formatted_json(add_street_df, sep=".")
format_json

[{'_id': 'DexsDsw39ss5NyhqQPUP',
  '_index_map': 0.0,
  'answers': {'contactnumber': {'ext': {'number': 9,
     'trunkline': 'Reception'}}}},
 {'_id': 'DexsDsw39ss5NyhqQPUP',
  '_index_map': 0.1,
  'answers': {'contactnumber': {'ext': {'number': 5,
     'trunkline': 'Operator'}}}},
 {'_id': 'DexsDsw39ss5NyhqQPUP',
  '_index_map': 1.0,
  'answers': {'contactnumber': {'ext': {'number': 23,
     'trunkline': 'Mobile Trunkline'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 0.0,
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Reception'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 0.0,
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Reception'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 1.0,
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': 'Mobile Trunkline'}}}},
 {'_id': 'aQMjmsDcwMbPs5NyhqQPUZ',
  '_index_map': 1.0,
  'answers': {'contactnumber': {'ext': {'number': 2,
     'trunkline': '

In [13]:
result = [
    {
        "standard": "119",
        "score": "0",
        "type": "assignment",
        "student": "4"
    },
    {
        "standard": "119",
        "score": "0",
        "type": "assignment",
        "student": "5"
    },
    {
        "standard": "118",
        "score": "0",
        "type": "assessment",
        "student": "4"
    }
]

In [14]:
def conf_to_nested_dict(keys, result):
    R = {}
    for record in result: 
        node = R
        for key in keys[:-1]:
            kv = record[key]
            next_node = node.get(kv, {})
            node[kv] = next_node
            node = next_node
        last_node = node.get(record[keys[-1]], [])
        last_node.append(record)
        node[record[keys[-1]]] = last_node


    return R

In [15]:
shit = conf_to_nested_dict(['standard','type'],result)
shit

{'119': {'assignment': [{'standard': '119',
    'score': '0',
    'type': 'assignment',
    'student': '4'},
   {'standard': '119', 'score': '0', 'type': 'assignment', 'student': '5'}]},
 '118': {'assessment': [{'standard': '118',
    'score': '0',
    'type': 'assessment',
    'student': '4'}]}}

In [16]:
add_street_df  = pd.DataFrame()
add_street_df = pd.read_csv( 'file/sample_output/answers.contactnumber.ext.csv') 
add_street_df = add_street_df.sort_values(['_id','_index_map']).reset_index(drop=True)
add_street_df

,_id,_index_map,answers.contactnumber.ext.number,answers.contactnumber.ext.trunkline
0,DexsDsw39ss5NyhqQPUP,0.0,9,Reception
1,DexsDsw39ss5NyhqQPUP,0.1,5,Operator
2,DexsDsw39ss5NyhqQPUP,1.0,23,Mobile Trunkline
3,aQMjmsDcwMbPs5NyhqQPUZ,0.0,2,Reception
4,aQMjmsDcwMbPs5NyhqQPUZ,0.0,2,Reception
5,aQMjmsDcwMbPs5NyhqQPUZ,1.0,2,Mobile Trunkline
6,aQMjmsDcwMbPs5NyhqQPUZ,1.0,2,Mobile Trunkline
7,aQMjmsDcwMbPs5NyhqQPUZ,1.1,3,Operator
8,aQMjmsDcwMbPs5NyhqQPUZ,1.1,3,Operator


In [17]:
src = "answers.contactnumber/0.ext/0.number:1"

In [18]:
w = {}
w['number'] = 1
w['trunkline'] = 'Operation'
w

{'number': 1, 'trunkline': 'Operation'}

In [19]:
z = {}
z['number'] = 2
z['trunkline'] = 'Mobile'
z

{'number': 2, 'trunkline': 'Mobile'}

In [20]:
d = {}

d.setdefault('ext', [None]*2)

d['ext'][0] = {}
d['ext'][0].update(w)

#d['ext'][0].update(y)
d['ext'][1] = {}
d['ext'][1].update(w)

d

{'ext': [{'number': 1, 'trunkline': 'Operation'},
  {'number': 1, 'trunkline': 'Operation'}]}

In [21]:
dz = {}
dz.setdefault('ext', [])

dz
dz['ext'].append({})
dz['ext'][0].update(w)

dz['ext'].append({})
dz['ext'][1].update(z)
dz

{'ext': [{'number': 1, 'trunkline': 'Operation'},
  {'number': 2, 'trunkline': 'Mobile'}]}

In [22]:
test = 'answers.contactnumber/0.ext/0.number:1'

index = 0
for keys in test.split('.'):
    print(keys)
    print(index)
    index += 1

test.split('.')[-1:]

answers
0
contactnumber/0
1
ext/0
2
number:1
3


['number:1']

In [23]:

dt = {}
testing = []
indexes = 0
dz = {}
for keys in test.split('.'):
    #dz = {}
    actual_value = {}
    #dz.setdefault('ext', [])
    
    #this is final item
    if ':' in keys: 
        print('KEYS: ', keys)
        #dt.setdefault(keys, {})
        value = (keys.split(sep=':')[-1:])
        new_key = (keys.split(sep=':')[0])
        actual_value[new_key] = value[0]
        testing.append(actual_value)
        
    elif '/' in keys: #this is list"
        #pass
        
        new_keyz = (keys.split(sep='/')[0])
        
        indexz = list((keys.split(sep='/')[-1:]))[0]
        dz.setdefault(new_keyz, [])

        dz[new_keyz].append({})
        
        testing.append(dz)
        #if bool(dt):
        print('INDEX:::', indexz)
        
        dz[new_keyz][int(indexz)].update(testing[indexes-1])
        del testing[indexes]
        #print(dt)
        print('---LIST-----')
        
        print(dz)
        #dt['contactnumber'][0].update({'ext': [{}]})
    else:
        foo = {}
        foo.setdefault(keys, {})
        testing.append(foo)
        
    indexes += 1
    #print('shit', testing[indexes-1])
    

print('-----_TEST_----')
#print(dz)
    


INDEX::: 0
---LIST-----
{'contactnumber': [{'answers': {}}]}
INDEX::: 0


IndexError: list assignment index out of range

In [ ]:
print('--ACTUAL VALUE-------')
print(actual_value)

In [ ]:
testing

## ROOT COPY

In [ ]:
testing[0]['answers'] = testing[1]

In [ ]:
testing[2]

## Non-root copy

In [ ]:
testing[1]['contactnumber'][0] = testing[2]
testing

In [ ]:
testing

## TEST ARRAY TESTING

In [ ]:
ixd = 1
testing_key = list(testing[len(testing)-(ixd+1)].keys())[0]
value_type = type(list(testing[len(testing)-(ixd+1)].values())[0])

testing[len(testing)-(ixd+1)][testing_key][0] = testing[len(testing)-ixd]

#testing[0][testing_key] = testing[len(testing)-ixd]

del testing[len(testing)-1]
testing

In [ ]:
ixd = 1
testing_key = list(testing[len(testing)-(ixd+1)].keys())[0]
value_type = type(list(testing[len(testing)-(ixd+1)].values())[0])

if type(list(testing[len(testing)-(ixd+1)].values())[0]) is list:
    testing[len(testing)-(ixd+1)][testing_key][0] = testing[len(testing)-ixd]
else:
    testing[0][testing_key] = testing[len(testing)-ixd]

del testing[len(testing)-1]
testing

In [ ]:
type(list(testing[0].values())[0])

In [ ]:
type(list(testing[len(testing)-2].values())[0])


In [ ]:

for item in testing:
    ixd = 1
    testing_key = list(testing[len(testing)-(ixd+1)].keys())[0]
    value_type = type(list(testing[len(testing)-(ixd+1)].values())[0])

    if type(list(testing[len(testing)-(ixd+1)].values())[0]) is list:
        testing[len(testing)-(ixd+1)][testing_key][0] = testing[len(testing)-ixd]
    else:
        testing[0][testing_key] = testing[len(testing)-ixd]
    #del testing[len(testing)-1]
    ixd += 1


In [ ]:
testing

In [ ]:
a = {}
a.setdefault('contactnumber', [None])

a['contactnumber'][0] = {}
a['contactnumber'][0].update(d)

a

In [ ]:
q = {}
q.setdefault('type', '')
q['type'] = 'Landline'

a['contactnumber'][0].update(q)
a

In [ ]:
json.dumps(a)

In [ ]:
xy = defaultdict(dict)

xy['contactnumber']['ext'] = 'foo'
#xy['contactnumber']['ext'] = 'bar'
xy